# Data Processing and Analysis

Data Processing is the most important and most time consuming component of the overall lifecycle of any Machine Learning project. 

In this notebook, we will analyze a dummy dataset to understand different issues we face with real world datasets and steps to handle the same.

## Import 

In [1]:
# import required libraries
import datetime
import random
from random import randrange

import numpy as np
import pandas as pd
from IPython.display import display
from sklearn import preprocessing

pd.options.mode.chained_assignment = None

## Dataset Utilities

In [2]:
def _random_date(start,date_count):
    """This function generates a random date based on params
    Args:
        start (date object): the base date
        date_count (int): number of dates to be generated
    Returns:
        list of random dates

    """
    current = start
    while date_count > 0:
        curr = current + datetime.timedelta(days=randrange(42))
        yield curr
        date_count-=1

In [3]:
def generate_sample_data(row_count=100):
    """This function generates a random transaction dataset
    Args:
        row_count (int): number of rows for the dataframe
    Returns:
        a pandas dataframe

    """

    # sentinels
    startDate = datetime.datetime(2016, 1, 1, 13)
    serial_number_sentinel = 1000
    user_id_sentinel = 5001
    product_id_sentinel = 101
    price_sentinel = 2000

    # base list of attributes
    data_dict = {
        'Serial No':
        np.arange(row_count) + serial_number_sentinel,
        'Date':
        np.random.permutation(
            pd.to_datetime([
                x.strftime("%d-%m-%Y")
                for x in _random_date(startDate, row_count)
            ]).date),
        'User ID':
        np.random.permutation(
            np.random.randint(0, row_count, size=int(row_count / 10)) +
            user_id_sentinel).tolist() * 10,
        'Product ID':
        np.random.permutation(
            np.random.randint(0, row_count, size=int(row_count / 10)) +
            product_id_sentinel).tolist() * 10,
        'Quantity Purchased':
        np.random.permutation(np.random.randint(1, 42, size=row_count)),
        'Price':
        np.round(
            np.abs(np.random.randn(row_count) + 1) * price_sentinel,
            decimals=2),
        'User Type':
        np.random.permutation(
            [chr(random.randrange(97, 97 + 3 + 1)) for i in range(row_count)])
    }

    # introduce missing values
    for index in range(int(np.sqrt(row_count))):
        data_dict['Price'][np.argmax(
            data_dict['Price'] == random.choice(data_dict['Price']))] = np.nan
        data_dict['User Type'][np.argmax(
            data_dict['User Type'] == random.choice(
                data_dict['User Type']))] = np.nan
        data_dict['Date'][np.argmax(
            data_dict['Date'] == random.choice(data_dict['Date']))] = np.nan
        data_dict['Product ID'][np.argmax(data_dict['Product ID'] == random.
                                          choice(data_dict['Product ID']))] = 0
        data_dict['Serial No'][np.argmax(data_dict['Serial No'] == random.
                                         choice(data_dict['Serial No']))] = -1
        data_dict['User ID'][np.argmax(data_dict['User ID'] == random.choice(
            data_dict['User ID']))] = -101

    # create data frame
    df = pd.DataFrame(data_dict)

    return df

## Generate Dataset

In [4]:
## Generate a dataset with 1000 rows
df = generate_sample_data(row_count=___)
df.shape

(1000, 7)

### Analyze generated Dataset

In [5]:
df.head()

,Serial No,Date,User ID,Product ID,Quantity Purchased,Price,User Type
0,1000,2016-01-22,-101,0,39,2571.49,n
1,1001,NaN,5683,877,3,2466.93,n
2,1002,NaN,5418,853,32,2323.22,n
3,1003,2016-05-01,5730,358,37,2812.33,n
4,1004,2016-01-20,5002,729,30,4634.88,n


In [6]:
def describe_dataframe(df=pd.DataFrame()):
    """This function generates descriptive stats of a dataframe
    Args:
        df (dataframe): the dataframe to be analyzed
    Returns:
        None

    """
    print("\n\n")
    print("*"*30)
    print("About the Data")
    print("*"*30)
    
    print("Number of rows::",df.shape[0])
    print("Number of columns::",df.shape[__])
    print("\n")
    
    print("Column Names::",df.columns.values.tolist())
    print("\n")
    
    print("Column Data Types::\n",df.dtypes)
    print("\n")
    
    print("Columns with Missing Values::",df.columns[df.isnull().any()].tolist())
    print("\n")
    
    print("Number of rows with Missing Values::",len(pd.isnull(df).any(1).nonzero()[0].tolist()))
    print("\n")
    
    print("Sample Indices with missing data::",pd.isnull(df).any(1).nonzero()[0].tolist()[0:5])
    print("\n")
    
    print("General Stats::")
    print(df.info())
    print("\n")
    
    print("Summary Stats::")
    print(df.describe())
    print("\n")
    
    print("Dataframe Sample Rows::")
    display(df.head(5))

In [7]:
describe_dataframe(df)




******************************
About the Data
******************************
Number of rows:: 1000
Number of columns:: 7


Column Names:: ['Serial No', 'Date', 'User ID', 'Product ID', 'Quantity Purchased', 'Price', 'User Type']


Column Data Types::
 Serial No               int64
Date                   object
User ID                 int64
Product ID              int64
Quantity Purchased      int64
Price                 float64
User Type              object
dtype: object


Columns with Missing Values:: ['Date', 'Price']


Number of rows with Missing Values:: 60


Sample Indices with missing data:: [1, 2, 6, 7, 13]


General Stats::
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
Serial No             1000 non-null int64
Date                  969 non-null object
User ID               1000 non-null int64
Product ID            1000 non-null int64
Quantity Purchased    1000 non-null int64
Price                 969 non-null float64

,Serial No,Date,User ID,Product ID,Quantity Purchased,Price,User Type
0,1000,2016-01-22,-101,0,39,2571.49,n
1,1001,NaN,5683,877,3,2466.93,n
2,1002,NaN,5418,853,32,2323.22,n
3,1003,2016-05-01,5730,358,37,2812.33,n
4,1004,2016-01-20,5002,729,30,4634.88,n


## Standardize Columns

In [8]:
# list all columns
print("Dataframe columns:\n{}".format(df.___.___()))

Dataframe columns:
['Serial No', 'Date', 'User ID', 'Product ID', 'Quantity Purchased', 'Price', 'User Type']


### Utility to Standardize Columns

In [9]:
def cleanup_column_names(df,rename_dict={},do_inplace=True):
    """This function renames columns of a pandas dataframe
       It converts column names to snake case if rename_dict is not passed. 
    Args:
        rename_dict (dict): keys represent old column names and values point to 
                            newer ones
        do_inplace (bool): flag to update existing dataframe or return a new one
    Returns:
        pandas dataframe if do_inplace is set to False, None otherwise

    """
    if not rename_dict:
        return df.rename(columns={col: col.___().replace(' ','_') 
                    for col in df.columns.values.tolist()}, 
                  inplace=_____)
    else:
        return df.rename(columns=rename_dict,inplace=do_inplace)

In [10]:
cleanup_column_names(df)

In [11]:
# Updated column names
print("Dataframe columns:\n{}".format(df.columns.tolist()))

Dataframe columns:
['serial_no', 'date', 'user_id', 'product_id', 'quantity_purchased', 'price', 'user_type']


## Basic Manipulation

### Sort basis specific attributes

In [12]:
# Ascending for Serial No and Descending for Price
display(df.sort_values(['serial_no', 'price'], 
                         ascending=[True, ___]).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
151,-1,2016-11-01,5979,220,20,4897.51,c
406,-1,2016-01-26,5467,873,10,4893.92,d
519,-1,2016-01-20,5461,850,12,4702.06,c
956,-1,2016-01-21,5996,1074,2,4664.97,d
595,-1,2016-01-30,5308,222,37,4590.35,c


### Reorder columns

In [14]:
display(df[['serial_no','date','user_id','user_type',
              'product_id','quantity_purchased','price']].head())

,serial_no,date,user_id,user_type,product_id,quantity_purchased,price
0,1000,2016-01-22,-101,n,0,39,2571.49
1,1001,NaN,5683,n,877,3,2466.93
2,1002,NaN,5418,n,853,32,2323.22
3,1003,2016-05-01,5730,n,358,37,2812.33
4,1004,2016-01-20,5002,n,729,30,4634.88


### Select Attributes

In [15]:
# Using Column Index
# print 10 values from column at index 3
print(df.iloc[:,3].values[0:10])

[  0 877 853 358 729 434 873 173 742 142]


In [16]:
# Using Column Name
# print 10 values of quantity purchased
print(df.quantity_purchased.values[0:10])

[39  3 32 37 30 31 27 36  5 34]


In [17]:
# Using Datatype
# print 10 values of columns with data type float
print(df.select_dtypes(include=['float64']).values[:10,0])

[2571.49 2466.93 2323.22 2812.33 4634.88 2659.72 1282.97 1683.48 3242.85
  175.81]


### Select Rows

In [19]:
# Using Row Index
display(df.iloc[[10,501,20]])

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
10,1010,2016-01-24,5073,808,19,2347.45,n
501,1501,2016-07-01,5683,877,2,545.09,b
20,1020,2016-01-30,5316,462,29,157.65,n


In [20]:
# Exclude specific rows
display(df.drop([0,24,51], axis=0).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
1,1001,NaN,5683,877,3,2466.93,n
2,1002,NaN,5418,853,32,2323.22,n
3,1003,2016-05-01,5730,358,37,2812.33,n
4,1004,2016-01-20,5002,729,30,4634.88,n
5,1005,2016-11-02,5173,434,31,2659.72,n


In [21]:
# Conditional Filtering
# Quantity_Purchased greater than 25
display(df[df.quantity_purchased __ 25].head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
0,1000,2016-01-22,-101,0,39,2571.49,n
2,1002,NaN,5418,853,32,2323.22,n
3,1003,2016-05-01,5730,358,37,2812.33,n
4,1004,2016-01-20,5002,729,30,4634.88,n
5,1005,2016-11-02,5173,434,31,2659.72,n


In [22]:
# Offset from Top
display(df[100:].head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
100,1100,2016-01-16,5700,388,11,5612.74,c
101,1101,2016-01-21,5683,877,10,5703.08,b
102,1102,NaN,5418,853,37,4826.85,d
103,1103,2016-01-27,5730,358,1,2936.60,a
104,1104,2016-03-01,5002,729,2,3383.50,a


In [23]:
# Offset from Bottom
display(df[-10:].head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
990,1990,2016-08-02,5371,633,4,3151.37,a
991,1991,2016-11-02,5636,994,27,3614.81,b
992,1992,2016-01-26,5508,562,8,1334.92,c
993,1993,2016-10-02,5898,904,26,5036.20,c
994,1994,2016-10-02,5442,1016,23,5191.03,c


### Type Casting

In [24]:
# Existing Datatypes
df.dtypes

serial_no               int64
date                   object
user_id                 int64
product_id              int64
quantity_purchased      int64
price                 float64
user_type              object
dtype: object

In [25]:
# Set Datatime as dtype for date column
df['date'] = pd.to_datetime(df.date)
print(df.dtypes)

serial_no                      int64
date                  datetime64[ns]
user_id                        int64
product_id                     int64
quantity_purchased             int64
price                        float64
user_type                     object
dtype: object


### Map/Apply Functionality

In [26]:
def expand_user_type(u_type):
    """This function maps user types to user classes
    Args:
        u_type (str): user type value
    Returns:
        (str) user_class value

    """
    if u_type in ['a','b']:
        return 'new'
    elif u_type == 'c':
        return 'existing'
    elif u_type == 'd':
        return 'loyal_existing'
    else:
        return 'error'

In [28]:
# Map User Type to User Class
df['user_class'] = df['user_type'].map(____)
display(df.tail())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class
995,-1,2016-08-01,5308,222,23,890.11,c,existing
996,1996,2016-01-28,5468,376,5,2660.33,c,existing
997,1997,2016-10-01,5632,1005,9,3380.72,b,new
998,1998,2016-03-02,5005,220,11,2629.37,c,existing
999,1999,2016-07-01,5487,420,4,1368.66,d,loyal_existing


In [29]:
# Apply: Using apply to get attribute ranges
display(df.select_dtypes(include=[np.number]).apply(lambda x: 
                                                        x.max()- x.____()))

serial_no             2000.0
user_id               6097.0
product_id            1084.0
quantity_purchased      40.0
price                 8722.4
dtype: float64

In [30]:
# Apply-Map: Extract Week from Date
df['purchase_week'] = df[['date']].applymap(lambda dt:dt.week 
                                                if not pd.isnull(dt.week) 
                                                else 0)

In [31]:
display(df.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
0,1000,2016-01-22,-101,0,39,2571.49,n,error,3
1,1001,NaT,5683,877,3,2466.93,n,error,0
2,1002,NaT,5418,853,32,2323.22,n,error,0
3,1003,2016-05-01,5730,358,37,2812.33,n,error,17
4,1004,2016-01-20,5002,729,30,4634.88,n,error,3


## Handle Missing Values

In [32]:
# Drop Rows with Missing Dates
df_dropped = df.dropna(subset=['date'])
display(df_dropped.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
0,1000,2016-01-22,-101,0,39,2571.49,n,error,3
3,1003,2016-05-01,5730,358,37,2812.33,n,error,17
4,1004,2016-01-20,5002,729,30,4634.88,n,error,3
5,1005,2016-11-02,5173,434,31,2659.72,n,error,44
8,1008,2016-01-27,5189,742,5,3242.85,n,error,4


In [33]:
# Filling missing price with mean price
df_dropped['price'].fillna(value=np.round(df.price.mean(),decimals=2),
                                inplace=True)

In [34]:
# Fill missing user types using values from previous row
df_dropped['user_type'].fillna(method='ffill',inplace=True)

## Handle Duplicates

In [35]:
# sample duplicates. Identify for serial_no
display(df_dropped[df_dropped.duplicated(subset=[____])].head())
print("Shape of df={}".format(df_dropped.shape))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
151,-1,2016-11-01,5979,220,20,4897.51,c,existing,44
287,-1,2016-01-21,5245,808,11,1280.13,a,new,3
291,-1,2016-01-18,5636,994,24,195.15,b,new,3
393,-1,2016-01-28,5898,904,23,4083.97,b,new,4
396,-1,2016-01-15,5468,376,28,1945.53,b,new,2


Shape of df=(969, 9)


In [36]:
# Drop Duplicates
df_dropped.drop_duplicates(subset=['serial_no'],inplace=True)
display(df_dropped.head())
print("Shape of df={}".format(df_dropped.shape))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
0,1000,2016-01-22,-101,0,39,2571.49,n,error,3
3,1003,2016-05-01,5730,358,37,2812.33,n,error,17
4,1004,2016-01-20,5002,729,30,4634.88,n,error,3
5,1005,2016-11-02,5173,434,31,2659.72,n,error,44
8,1008,2016-01-27,5189,742,5,3242.85,n,error,4


Shape of df=(940, 9)


In [37]:
# Remove rows which have less than 3 attributes with non-missing data
display(df.dropna(thresh=___).head())
print("Shape of df={}".format(df.dropna(thresh=____).shape))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
0,1000,2016-01-22,-101,0,39,2571.49,n,error,3
1,1001,NaT,5683,877,3,2466.93,n,error,0
2,1002,NaT,5418,853,32,2323.22,n,error,0
3,1003,2016-05-01,5730,358,37,2812.33,n,error,17
4,1004,2016-01-20,5002,729,30,4634.88,n,error,3


Shape of df=(1000, 9)


## Handle Categoricals

### One Hot Encoding

In [38]:
display(pd.get_dummies(df,columns=['user_type']).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_class,purchase_week,user_type_a,user_type_b,user_type_c,user_type_d,user_type_n
0,1000,2016-01-22,-101,0,39,2571.49,error,3,0,0,0,0,1
1,1001,NaT,5683,877,3,2466.93,error,0,0,0,0,0,1
2,1002,NaT,5418,853,32,2323.22,error,0,0,0,0,0,1
3,1003,2016-05-01,5730,358,37,2812.33,error,17,0,0,0,0,1
4,1004,2016-01-20,5002,729,30,4634.88,error,3,0,0,0,0,1


### Label Encoding

In [39]:
type_map = {'a': 0, 'b': 1, 'c': 2, 'd': 3, np.NAN: -1}
df['encoded_user_type'] = df.user_type.map(____)
display((df.tail()))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week,encoded_user_type
995,-1,2016-08-01,5308,222,23,890.11,c,existing,31,2.0
996,1996,2016-01-28,5468,376,5,2660.33,c,existing,4,2.0
997,1997,2016-10-01,5632,1005,9,3380.72,b,new,39,1.0
998,1998,2016-03-02,5005,220,11,2629.37,c,existing,9,2.0
999,1999,2016-07-01,5487,420,4,1368.66,d,loyal_existing,26,3.0


## Handle Numerical Attributes

### Min-Max Scalar

In [40]:
df_normalized = df.dropna().copy()
min_max_scaler = preprocessing.______()
np_scaled = min_max_scaler.fit_transform(df_normalized['price'].values.reshape(-1,1))
df_normalized['price'] = np_scaled.reshape(-1,1)

In [41]:
display(df_normalized.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week,encoded_user_type
22,1022,2016-01-01,5676,1067,19,0.214495,d,loyal_existing,53,3.0
28,1028,2016-04-01,5973,104,12,0.136061,d,loyal_existing,13,3.0
30,1030,2016-01-25,5207,286,23,0.257740,d,loyal_existing,4,3.0
31,1031,2016-08-01,5688,583,31,0.232022,d,loyal_existing,31,3.0
33,1033,2016-06-02,5139,947,6,0.274444,a,new,22,0.0


### Robust Scalar

In [42]:
df_normalized = df.dropna().copy()
robust_scaler = preprocessing.RobustScaler()
rs_scaled = robust_scaler._______(df_normalized['quantity_purchased'].values.reshape(-1,1))
df_normalized['quantity_purchased'] = rs_scaled.reshape(-1,1)

In [43]:
display(df_normalized.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week,encoded_user_type
22,1022,2016-01-01,5676,1067,-0.142857,1879.91,d,loyal_existing,53,3.0
28,1028,2016-04-01,5973,104,-0.476190,1196.08,d,loyal_existing,13,3.0
30,1030,2016-01-25,5207,286,0.047619,2256.95,d,loyal_existing,4,3.0
31,1031,2016-08-01,5688,583,0.428571,2032.72,d,loyal_existing,31,3.0
33,1033,2016-06-02,5139,947,-0.761905,2402.58,a,new,22,0.0


## Group-By

In [44]:
# Group By certain attributes user_class and get sum of quantity_purchased
print(df.groupby(['user_class'])['quantity_purchased'].____())

user_class
error               689
existing           5575
loyal_existing     4670
new               10347
Name: quantity_purchased, dtype: int64


In [45]:
# Aggregate Functions. Sum, Mean and Non Zero Row Count
display(
    df.groupby(['user_class'])['quantity_purchased'].agg(
        [np.____, np.____, np.count_nonzero]))

,sum,mean,count_nonzero
user_class,,,
error,689,22.225806,31
existing,5575,22.662602,246
loyal_existing,4670,20.304348,230
new,10347,20.987830,493


In [46]:
# Aggregate Functions specific to columns
display(df.groupby(['user_class','user_type']).agg({'price':np.mean,
                                                        'quantity_purchased':np.max}))

price  quantity_purchased
user_class     user_type                                 
error          n          2644.041290                  39
existing       c          2374.785679                  41
loyal_existing d          2450.062273                  41
new            a          2107.919437                  41
               b          2449.754549                  41

In [47]:
# Multiple Aggregate Functions
display(
    df.groupby(['user_class', 'user_type']).agg({
        'price': {
            'total_price': np.sum,
            'mean_price': np.mean,
            'variance_price': np.std,
            'count': np.count_nonzero
        },
        'quantity_purchased': np.sum
    }))

/Users/rbali2/anaconda/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


price                                     \
                         total_price   mean_price variance_price  count   
user_class     user_type                                                  
error          n            81965.28  2644.041290    1945.440890   31.0   
existing       c           577072.92  2374.785679    1628.365448  246.0   
loyal_existing d           539013.70  2450.062273    1622.086894  230.0   
new            a           486929.39  2107.919437    1549.298323  241.0   
               b           597740.11  2449.754549    1775.230970  252.0   

                         quantity_purchased  
                                        sum  
user_class     user_type                     
error          n                        689  
existing       c                       5575  
loyal_existing d                       4670  
new            a                       5208  
               b                       5139

## Pivot Tables

In [48]:
display(df.pivot_table(index='date', columns='user_type', 
                         values='price',aggfunc=np.mean))

user_type,a,b,c,d,n
date,,,,,
2016-01-01,2252.256667,3039.420000,4030.830000,2609.781667,2096.495000
2016-01-02,2348.065000,2234.445000,3417.777500,2462.582500,915.280000
2016-01-13,1112.392500,3181.868750,1750.672500,2505.297778,NaN
2016-01-14,3103.557500,3106.012000,3687.740000,2936.520000,NaN
2016-01-15,1915.987143,1942.046000,1614.273333,3527.080000,NaN
2016-01-16,2910.938889,4032.250000,3273.530000,1868.254000,NaN
2016-01-17,2207.140000,1882.300000,526.880000,841.180000,175.810000
2016-01-18,1707.212857,2471.696667,1531.380000,2284.608333,NaN
2016-01-19,2541.122000,1534.025000,2259.712000,1358.688333,NaN


## Stacking

In [50]:
print(df.stack())

0    serial_no                            1000
     date                  2016-01-22 00:00:00
     user_id                              -101
     product_id                              0
     quantity_purchased                     39
     price                             2571.49
     user_type                               n
     user_class                          error
     purchase_week                           3
1    serial_no                            1001
     user_id                              5683
     product_id                            877
     quantity_purchased                      3
     price                             2466.93
     user_type                               n
     user_class                          error
     purchase_week                           0
2    serial_no                            1002
     user_id                              5418
     product_id                            853
     quantity_purchased                     32
     price   